# Optimal Lag Selection based on MRMR

In [1]:
# Description: 

# Part I: MRMR criterion selection

# Author: Shilin Zhang

# Problem: Finding a robust way of choosing an optimal lag on mutiple kinds of time series regression models

# Method: Minimize Redundancy Maxmize Relevance, Genetic Algorithm, Forwardstep selection

# Measurement: correlation coefficient, mutual information, partial correlation

# Related papers: 
# Comprehensive study of feature selection methods to solve multicollinearity problem according to evaluation criteria - Alexandr Katrutsa
# Effective Global Approaches for Mutual Information Based Feature Selection - Nguyen Xuan Vinh

# Program Start Date: June 1st 2022

In [2]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mutual_info_score
import pingouin as pg
from functools import reduce

from numpy.random import randint
from numpy.random import rand
np.random.seed(10)

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load the data set from local file

startdate = '2012-04-01'
enddate = '2022-06-01'
datasource = 'SPY'

data = yf.download(datasource,start = startdate,end = enddate)

[*********************100%***********************]  1 of 1 completed


In [4]:
target = 'Close'
train_test_ratio = 0.7

In [5]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-04-02,140.639999,142.210007,140.360001,141.839996,116.762009,151741100
2012-04-03,141.639999,141.880005,140.429993,141.259995,116.284554,155806700
2012-04-04,140.220001,140.339996,139.339996,139.860001,115.132072,146896000
2012-04-05,139.380005,140.199997,139.259995,139.789993,115.074417,137439400
2012-04-09,138.029999,139.839996,137.839996,138.220001,113.782036,127555900
...,...,...,...,...,...,...
2022-05-24,392.559998,395.149994,386.959991,393.890015,392.195831,91448800
2022-05-25,392.309998,399.450012,391.890015,397.369995,395.660858,91472900
2022-05-26,398.670013,407.040009,398.450012,405.309998,403.566711,82168300


In [6]:
# Function spliting train test set 

def train_test_split(data, train_test_ratio):
    
    split_point = round(data.shape[0] * train_test_ratio)
    training_data = data.iloc[0:split_point,:]
    testing_data = data.iloc[split_point: ,:]
    
    return training_data, testing_data

### Data Preprocessing

In [7]:
# Shift the target to avoid the data leakage

def data_shift(train,target):
    data_shifted = train.shift()
    data_preprocessed = pd.concat([data_shifted,train[target]], axis=1)
    data_preprocessed.dropna(inplace=True)
    return data_preprocessed

In [8]:
# Parameter Setting

# maximum lag
max_lag = 30

# length of an individual
n_d = data.shape[1] * max_lag

# number of generation
n_g = 300
# number of population
n_p = 100
# crossover prob
p_c = 0.85
# mutation prob
p_m = 0.05

# parameter for negentropy estimation
a_1 = 1.5

In [9]:
# Construct the feature space

def feature_space_generator(data, max_lag):
    X = np.empty((data.shape[0],0))
    y = pd.DataFrame(data.iloc[:,-1])
    for lag in range(0,max_lag):
       X = np.concatenate((X, data.iloc[:,:-1].shift(lag)), axis = 1) 
    Xy = pd.DataFrame(np.concatenate((X, y), axis = 1)).dropna()
    X = pd.DataFrame(Xy.iloc[:,:-1])
    y = pd.DataFrame(Xy.iloc[:,-1])
    return X, y

In [10]:
# Function normalizing of sim and rel matrix

def normalization(Q):
    df = pd.DataFrame(Q)
    cols = df.columns.values.tolist()
    df_new = df.divide(df[cols].values.sum())
    return df_new

In [11]:
def opt_problem_generator(X, y, sim, rel):
    if sim == 'correl':
        Q = abs(np.corrcoef(X.T))
    elif sim == 'mi':
        Q = np.zeros((X.shape[1],X.shape[1]))
        for i in range(0,X.shape[1]):
            for j in range(0,X.shape[1]):
                if(i<j):
                    Q[i,j] = mutual_info_score(X.iloc[:,i],X.iloc[:,j])
                elif(i>j):
                    Q[i,j] = Q[j,i]
    if rel == 'correl':
        b = abs(pd.DataFrame(np.concatenate((X,y),axis = 1)).corr().iloc[:-1,-1])
    elif rel == 'mi':
        b = np.zeros((X.shape[1],1))
        for i in range(0,X.shape[1]):
            b[i,0] = mutual_info_score(X.iloc[:,i],y.iloc[:,0])
    elif rel == 'pcor':
        b = abs(pd.DataFrame(np.concatenate((X,y),axis = 1)).pcorr()).iloc[:-1,-1]
    
    #Q = np.array(normalization(Q))
    b = np.array(normalization(b)).tolist()
    b = reduce(lambda x, y: x + y, b)
    b = np.array(b)
    return Q,b

In [12]:
# Genetic algorithm search for optimization problem

# Objective function

# x is the binary/gene list shows the presence or absence of the feature in feature subset

def obj(x, Q, b):
    a = pd.DataFrame(x)
    sim_value = np.dot(np.dot(a.T,Q),a)[0,0]
    rel_value = np.dot(b.T,a)
    score = sim_value - rel_value
    return score[0]

In [13]:
# Selection

def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

In [14]:
# Crossover two parents to create two children

def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

In [15]:
# Mutation operator

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

In [16]:
# Genetic algorithm

def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut, Q, b):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(pop[0],Q, b)
    # enumerate generations
    for gen in range(n_iter):
        # print("iteration: " + str(gen + 1))
        # evaluate all candidates in the population
        scores = [objective(c,Q, b) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print("iteration: %d, number of features: %d, score: %.3f" %(gen + 1, sum(pop[i]), scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

In [17]:
sim_list = ['correl','mi']
rel_list = ['correl','mi','pcor']

# Function selecting laged features based on mutiple criterions

def multi_lag_selection(sim_list,rel_list,X,y):
    best_list = []
    score_list = []
    for sim in sim_list:
        for rel in rel_list:
            print(sim,rel)
            Q,b =  opt_problem_generator(X, y, sim, rel)
            best, score = genetic_algorithm(obj, n_d, n_g, n_p, p_c, p_m, Q, b)
            best_list.append(best)
            score_list.append(score)
    return best_list,score_list

In [18]:
# Transfer bitstring of integer values to feature subset

def feature_subset_generator(X,gene_list):
    result = np.empty((X.shape[0],0))
    n = 0
    for i in gene_list:
        if i == 1:
            result = np.concatenate((result, pd.DataFrame(X.iloc[:,n])), axis = 1) 
        n  = n + 1
    return pd.DataFrame(result)

In [19]:
# Prepare training or testing dataset

def generate_list(X,y, best_list):
    train_list = []
    for i in best_list:
        subset = feature_subset_generator(X,i)
        train = pd.DataFrame(np.concatenate((subset, y), axis = 1))
        train_list.append(train)
    return train_list

In [20]:
# import necessary packages

import matplotlib.pyplot as plt
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import math

import warnings
warnings.filterwarnings("ignore")

In [21]:
def linear_regression_result(train_list,test_list):
    print('Result of Linear Regression')
    RMSE_all = []
    for i in range(0,len(train_list)):
        print('Criterion ' + str(i+1) + ': ')
        train = train_list[i]
        test = test_list[i]
            
        train_x = train.iloc[:,:(train.shape[1] - 1)]
        train_y = train.iloc[:,-1:]
            
        test_x = test.iloc[:,:(test.shape[1] - 1)]
        test_y = test.iloc[:,-1:]
            
        regressor = LinearRegression()
        model = TransformedTargetRegressor(regressor= regressor,
                                        transformer = MinMaxScaler()
                                        ).fit(train_x,train_y)
        y_pred = model.predict(test_x)
            
        RMSE = (math.sqrt(mean_squared_error(y_pred, test_y)))
        print(RMSE)
        RMSE_all.append(RMSE)
        
    print(min(RMSE_all))

    print(' ')
    return RMSE_all

In [22]:
def random_forest_result(train_list,test_list):
    print('Result of Random Forest')
    RMSE_all = []
    for i in range(0,len(train_list)):
        print('Criterion ' + str(i+1) + ': ')
        train = train_list[i]
        test = test_list[i]
            
        train_x = train.iloc[:,:(train.shape[1] - 1)]
        train_y = train.iloc[:,-1:]
            
        test_x = test.iloc[:,:(test.shape[1] - 1)]
        test_y = test.iloc[:,-1:]
            
        regressor = RandomForestRegressor()
        model = TransformedTargetRegressor(regressor= regressor,
                                        transformer = MinMaxScaler()
                                        ).fit(train_x,train_y)
        y_pred = model.predict(test_x)
            
        RMSE = (math.sqrt(mean_squared_error(y_pred, test_y)))
        print(RMSE)
        RMSE_all.append(RMSE)
        
    print(min(RMSE_all))
    print(' ')
    return RMSE_all

In [23]:
def elastic_net_result(train_list,test_list):
    print('Result of elastic net')
    RMSE_all = []
    for i in range(0,len(train_list)):
        print('Criterion ' + str(i+1) + ': ')

        train = train_list[i]
        test = test_list[i]
            
        train_x = train.iloc[:,:(train.shape[1] - 1)]
        train_y = train.iloc[:,-1:]
            
        test_x = test.iloc[:,:(test.shape[1] - 1)]
        test_y = test.iloc[:,-1:]
            
        model = ElasticNet()
        # define grid
        grid = dict()
        grid['alpha'] = [0.0, 1.0, 10.0, 100.0]
        grid['l1_ratio'] = [0.1,0.5,0.7,0.9,0.95,0.99,1]
        # define search
        search = GridSearchCV(model, grid, scoring='neg_mean_squared_error')
        best = search.fit(train_x,train_y)
        best_model = ElasticNet(l1_ratio = best.best_params_.get('l1_ratio'), alpha = best.best_params_.get('alpha')).fit(train_x,train_y)
            
        y_pred = best_model.predict(test_x)
        RMSE = (math.sqrt(mean_squared_error(y_pred, test_y)))
        print(RMSE)
        RMSE_all.append(RMSE)
        
    print(min(RMSE_all))
    print(' ')
    return RMSE_all

In [24]:
def svr_result(train_list,test_list):
    print('Result of SVR')
    RMSE_all = []
    for i in range(0,len(train_list)):
        print('Criterion ' + str(i+1) + ': ')
        train = train_list[i]
        test = test_list[i]
            
        train_x = train.iloc[:,:(train.shape[1] - 1)]
        train_y = train.iloc[:,-1:]
            
        test_x = test.iloc[:,:(test.shape[1] - 1)]
        test_y = test.iloc[:,-1:]
            
        regressor = SVR(kernel='rbf')
        model = TransformedTargetRegressor(regressor= regressor,
                                        transformer = MinMaxScaler()
                                        ).fit(train_x,train_y)
        y_pred = model.predict(test_x)
            
        RMSE = (math.sqrt(mean_squared_error(y_pred, test_y)))
        print(RMSE)
        RMSE_all.append(RMSE)
        
    print(min(RMSE_all))
    print(' ')
    return RMSE_all

In [25]:
def GALS(data,target):
    
    training_data,testing_data = train_test_split(data, 0.7)
    
    train_preprocessed = data_shift(training_data,target)
    test_preprocessed = data_shift(testing_data,target)
    
    X, y = feature_space_generator(train_preprocessed, max_lag)
    test_X, test_y = feature_space_generator(test_preprocessed, max_lag)
    
    best_list,score_list = multi_lag_selection(sim_list,rel_list,X,y)
    
    train_list = generate_list(X, y, best_list)
    test_list = generate_list(test_X, test_y, best_list)
    
    RMSE_LR = linear_regression_result(train_list,test_list)
    RMSE_RF = random_forest_result(train_list,test_list)
    RMSE_EN = elastic_net_result(train_list,test_list)
    RMSE_SVR = svr_result(train_list,test_list)
    
    return RMSE_LR, RMSE_RF,RMSE_EN,RMSE_SVR

In [26]:
RMSE_LR, RMSE_RF,RMSE_EN,RMSE_SVR= GALS(data,target)

correl correl
iteration: 1, number of features: 79, score: 4968.263
iteration: 1, number of features: 79, score: 4809.919
iteration: 1, number of features: 71, score: 4063.565
iteration: 4, number of features: 69, score: 3658.786
iteration: 5, number of features: 67, score: 3243.590
iteration: 6, number of features: 65, score: 3200.759
iteration: 7, number of features: 56, score: 2547.110
iteration: 10, number of features: 56, score: 2542.245
iteration: 11, number of features: 60, score: 2508.007
iteration: 11, number of features: 54, score: 2347.374
iteration: 11, number of features: 55, score: 2234.674
iteration: 12, number of features: 51, score: 1871.922
iteration: 15, number of features: 50, score: 1747.302
iteration: 16, number of features: 49, score: 1665.373
iteration: 22, number of features: 50, score: 1625.726
iteration: 26, number of features: 46, score: 1475.172
iteration: 27, number of features: 44, score: 1445.796
iteration: 29, number of features: 42, score: 1385.983
ite

iteration: 33, number of features: 45, score: 14493.322
iteration: 37, number of features: 44, score: 13832.659
iteration: 38, number of features: 42, score: 12585.125
iteration: 44, number of features: 40, score: 11405.487
iteration: 62, number of features: 39, score: 10845.592
iteration: 135, number of features: 39, score: 10826.019
iteration: 144, number of features: 38, score: 10310.054
iteration: 147, number of features: 38, score: 10308.290
iteration: 148, number of features: 37, score: 9776.338
iteration: 181, number of features: 37, score: 9743.489
iteration: 201, number of features: 37, score: 9736.806
iteration: 224, number of features: 34, score: 8214.353
Result of Linear Regression
Criterion 1: 
7.667561752409476
Criterion 2: 
4.882208272283286
Criterion 3: 
6.344496637066148
Criterion 4: 
4.811976163596782
Criterion 5: 
8.989843858671726
Criterion 6: 
7.900788888644357
4.811976163596782
 
Result of Random Forest
Criterion 1: 
106.99350088592182
Criterion 2: 
103.7694884584